<a href="https://colab.research.google.com/github/farshidbalan/FluentPython/blob/master/Chapter13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operator Overloading: Doing It Right

From Chapter10, we have the Vector class. We want an output like:

```
v1 = Vector([3, 4, 5, 6])
 
v3 = Vector([1, 2])

v1 + v3

Vector([4.0, 6.0, 5.0, 6.0])
```


In [0]:
from array import array
import reprlib
import itertools
class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    # The self._components instance “protected” attribute will hold an array 
    # with the Vector components.
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation of 
    # self._components (e.g., array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])).
    components = reprlib.repr(self._components)
    
    # Remove the array('d', prefix and the trailing ) before plugging the string 
    # into a Vector constructor call.
    components = components[components.find('['): -1]
    return f'Vector({components})'
  
  def __bytes__(self):
    # Build a bytes object directly from self._components
    return bytes([ord(self.typecode)]) + bytes(self._components)
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __boll__(self):
    return bool(abs(self))
  
  def __add__(self, other):
    pairs = itertools.zip_longest(self, other, fillvalue=0.0)  # pairs is a generator that will produce tuples (a, b) where a is from self, and
                                                               #b is from other. If self and other have different lengths, fillvalue is used to
                                                               # supply the missing values for the shortest iterable.
        
    return Vector(a + b for a, b in pairs)    # A new Vector is built from a generator expression producing one sum for each item in pairs.                
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    #  we pass the memoryview directly to the constructor
    return cls(memv)

In [0]:
v1 = Vector([1, 2, 3])
v2 = Vector([3, 4, 5, 3])
v1 + v2

Vector([4.0, 6.0, 8.0, 3.0])

### Remark

Very interesting to see that we return the object type in the  \_\_add\_\_ method



```
def __add__(self, other):
    pairs = itertools.zip_longest(self, other, fillvalue=0.0) 
    return Vector(a + b for a, b in pairs) 
        
```



### Remark

To support operations involving objects of different types, Python implements a special
dispatching mechanism for the infix operator special methods. Given an expression a+ b, the interpreter will perform these steps:
1. If a has \_\_add\_\_, call a. \_\_add \_\_(b) and return result unless it’s NotImplemented.
2. If a doesn’t have  \_\_add \_\_, or calling it returns NotImplemented, check if b has
 \_\_radd \_\_, then call b. \_\_radd \_\_(a) and return result unless it’s NotImplemented.
3. If b doesn’t have  \_\_radd \_\_, or calling it returns NotImplemented, raise TypeError
with an unsupported operand types message

In [0]:
# Example 13-6. Vector.__add__ take #1 fails with non-Vector left operands
v1 = Vector([3, 4, 5])
(10, 20, 30) + v1


TypeError: ignored

Often, \_\_radd\_\_ can be as simple as that: just invoke the proper operator, therefore
delegating to \_\_add\_\_ in this case. This applies to any commutative operator; + is commutative when dealing with numbers or our vectors, but it’s not commutative when
concatenating sequences in Python.

In [0]:
def __add__(self, other): #
  pairs = itertools.zip_longest(self, other, fillvalue=0.0)
  return Vector(a + b for a, b in pairs)

def __radd__(self, other):
  return self + other

## Overloading * for Scalar Multiplication

In [0]:
# Example 13-11. vector_v7.py: operator * methods added

from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other
      
    def __mul__(self, scalar):
      if isinstance(scalar, numbers.Real): # If scalar is an instance of a numbers.Real subclass, 
                                           # create new Vector with multiplied component values
        return Vector(n * scalar for n in self)
      else:
        return NotImplemented
      
    def __rmul__(self, scalar):
      return self * scalar  # In this example, __rmul__ works fine by just performing self * scalar, delegating to the __mul__ method
      

In [2]:
v1 = Vector([1.0, 3.0, 5.0])
14 * v1

Vector([14.0, 42.0, 70.0])

In [3]:
v1 * True

Vector([1.0, 3.0, 5.0])

In [5]:
from fractions import Fraction
v1 * Fraction(1, 3)

Vector([0.3333333333333333, 1.0, 1.6666666666666665])

## Further Reading
Operator overloading is one area of Python programming where isinstance tests are
common. In general, libraries should leverage dynamic typing—to be more flexible—
by avoiding explicit type tests and just trying operations and then handling the except
opening the door for working with objects regardless of their types, as long as
they support the necessary operations.